## 데이터를 로드하는 방법과 미니 배치 경사하강법(Minibatch Gradient Descent)에 대해서 학습한다.

### 1. 미니 배치와 배치 크기

In [1]:
import torch

In [2]:
x_train = torch.FloatTensor([[73, 80, 75],
                             [93, 88, 93],
                             [89, 91, 90],
                             [96, 98, 100],
                             [73, 66, 70]])
y_train = torch.FloatTensor([[152], [185], [180], [196], [142]])


위와 같은 데이터는 모델 학습에는 아쉬운 양이다.   
데이터가 수십만개는 필요하다. 그렇다면 빠르게 많은 계산량을 필요로 한다.  
그렇기 때문에 전체 데이터를 더 작은 단위로 나누어서 학습하는 개념이 필요하다.  
이 단위를 미니 배치(Mini Batch)라고 한다.

에포크는 전체 훈련 데이터가 학습에 한 번 사용된 주기를 말한다.  
위의 경우, 미니 배치의 개수만큼 경사 하강법을 수행해야 1 에포크가 된다.  
미니 배치의 크기에 따라 미니 배치의 갯수도 달라지겠다.  
미니 배치의 크기를, batch size라고 한다.

전체 데이터에 대해서 한 번에 경사 하강법을 수행하는 방법을 '배치 경사하강법',    
미니 배치 단위로 경사 하강법을 수행하는 방법을 '미니 배치 경사 하강법'이라고 부른다.

배치 경사하강법은, 전체 데이터를 사용하므로 가중치값이 최적값에 수렴하는 과정이 매우 안정적이나, 계산량이 너무나 많다.  
미니 배치 경사하강법은 전체 데이터의 일부만을 보고 수행하므로, 최적값에 수렴하는 과정이 불안할 수는 있으나, 속도가 빠르다.  

배치 크기는 보통 2의 제곱수를 사용한다.  
CPU와 GPU의 메모리가 2의 배수이므로 배치크기가 2의 제곱수일 때 데이터 송수신의 효율을 높일 수 있기 때문이다.

### 2. 이터레이션(Iteration)

이터레이션이란, 가중치 W와 편향 b가 업데이트되는 횟수를 말한다.   
Total data가 2000, Batch size가 200이라고 해보자.  
미니 배치 경사하강법에서, 미니 배치의 총 갯수는 10개가 될 것이고,  
이터레이션 또한 10이 될 것이다.


### 3. 데이터 로드하기(Data Load)

파이토치는 데이터를 더 쉽게 다룰 수 있도록 데이터셋(Dataset), 데이터로더(DataLoader)를 제공한다.   
이를 사용하면 미니 배치학습, 데이터 셔플(shuffle), 병렬 처리까지 간단히 수행할 수 있다.  
기본적인 사용방법은 Dataset을 정의하고, 이를 DataLoader에 전달하는 것이다.

Dataset을 커스텀할 수 있으나, 여기서는 TensorDataset을 사용해본다.

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader

In [5]:
x_train  =  torch.FloatTensor([[73,  80,  75], 
                               [93,  88,  93], 
                               [89,  91,  90], 
                               [96,  98,  100],   
                               [73,  66,  70]])  
y_train  =  torch.FloatTensor([[152],  [185],  [180],  [196],  [142]])

In [6]:
dataset = TensorDataset(x_train,y_train)

데이터로더의 인자는 기본적으로 2개이다.  
하나는 데이터셋, 다른 하나는 미니배치의 크기이다.  
상술한 바, 미니배치 크기는 2의 배수를 보통 사용한다.

추가적으로 shuffle 인자도 넣는다.  
shuffle = True면, Epoch마다 데이터셋을 섞어서 데이터가 학습되는 순서를 바꾼다.  
모델이 문제 순서 자체에 익숙해지는 것을 방지할 수 있다.

In [12]:
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

이제 모델과 옵티마이저를 설계한다.

In [13]:
model = nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(),lr=1e-5)

In [14]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        x_train, y_train = samples

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 35760.527344
Epoch    0/20 Batch 2/3 Cost: 6350.637695
Epoch    0/20 Batch 3/3 Cost: 1370.023193
Epoch    1/20 Batch 1/3 Cost: 1317.747070
Epoch    1/20 Batch 2/3 Cost: 179.465134
Epoch    1/20 Batch 3/3 Cost: 128.683441
Epoch    2/20 Batch 1/3 Cost: 27.523636
Epoch    2/20 Batch 2/3 Cost: 6.914958
Epoch    2/20 Batch 3/3 Cost: 6.549057
Epoch    3/20 Batch 1/3 Cost: 1.087140
Epoch    3/20 Batch 2/3 Cost: 0.442824
Epoch    3/20 Batch 3/3 Cost: 0.237659
Epoch    4/20 Batch 1/3 Cost: 0.909551
Epoch    4/20 Batch 2/3 Cost: 0.548521
Epoch    4/20 Batch 3/3 Cost: 0.084948
Epoch    5/20 Batch 1/3 Cost: 0.351038
Epoch    5/20 Batch 2/3 Cost: 0.745874
Epoch    5/20 Batch 3/3 Cost: 0.105970
Epoch    6/20 Batch 1/3 Cost: 0.643967
Epoch    6/20 Batch 2/3 Cost: 0.289566
Epoch    6/20 Batch 3/3 Cost: 0.001647
Epoch    7/20 Batch 1/3 Cost: 0.195470
Epoch    7/20 Batch 2/3 Cost: 0.563983
Epoch    7/20 Batch 3/3 Cost: 0.805244
Epoch    8/20 Batch 1/3 Cost: 0.592169
Epoch  

### 4. 커스텀 데이터셋 (Custom Dataset) 

torch.utils.dats.Dataset을 상속받아 직접 커스텀 데이터셋을 만드는 경우도 있다.  
torch.utils.data.Dataset은 파이토치에서 데이터셋을 제공하는 추상 클래스이다.

In [ ]:
# 기본적인 뼈대
class CustomDataset(torch.utils.data.Dataset):
    def __init__(self):
        # 데이터셋의 전처리를 해주는 부분
        pass

    def __len__(self):
        # 데이터셋의 길이. 즉, 총 샘플의 수를 적어주는 부분
        pass

    def __getitem__(self, idx):
        # 데이터셋에서 특정 1개의 샘플을 가져오는 함수
        pass

### 5. 커스텀 데이터셋으로 선형 회귀 구현하기

In [16]:
import torch
import torch.nn.functional as F
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

In [17]:
# Dataset 상속
class CustomDataset(Dataset):
    def __init__(self):
        self.x_data = [[73, 80, 75],
                   [93, 88, 93],
                   [89, 91, 90],
                   [96, 98, 100],
                   [73, 66, 70]]
        self.y_data = [[152], [185], [180], [196], [142]] 

    # 총 데이터의 개수를 리턴
    def __len__(self):
        return len(self.x_data)
    
    # 인덱스를 입력받아 그에 맵핑되는 입출력 데이터를 파이토치의 Tensor 형태로 리턴
    def __getitem__(self, idx):
        x = torch.FloatTensor(self.x_data[idx])
        y = torch.FloatTensor(self.y_data[idx])
        return x, y

In [18]:
dataset = CustomDataset()
dataloader = DataLoader(dataset, batch_size=2, shuffle=True)

In [19]:
model = torch.nn.Linear(3,1)
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-5)

In [20]:
nb_epochs = 20
for epoch in range(nb_epochs + 1):
    for batch_idx, samples in enumerate(dataloader):
        # print(batch_idx)
        # print(samples)
        x_train, y_train = samples

        # H(x) 계산
        prediction = model(x_train)

        # cost 계산
        cost = F.mse_loss(prediction, y_train)

        # cost로 H(x) 계산
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        print('Epoch {:4d}/{} Batch {}/{} Cost: {:.6f}'.format(
        epoch, nb_epochs, batch_idx+1, len(dataloader),
        cost.item()
        ))

Epoch    0/20 Batch 1/3 Cost: 85282.453125
Epoch    0/20 Batch 2/3 Cost: 14186.439453
Epoch    0/20 Batch 3/3 Cost: 11486.381836
Epoch    1/20 Batch 1/3 Cost: 1625.456909
Epoch    1/20 Batch 2/3 Cost: 566.051514
Epoch    1/20 Batch 3/3 Cost: 147.327728
Epoch    2/20 Batch 1/3 Cost: 55.996269
Epoch    2/20 Batch 2/3 Cost: 22.234821
Epoch    2/20 Batch 3/3 Cost: 1.620827
Epoch    3/20 Batch 1/3 Cost: 0.151190
Epoch    3/20 Batch 2/3 Cost: 8.265091
Epoch    3/20 Batch 3/3 Cost: 7.487826
Epoch    4/20 Batch 1/3 Cost: 2.591959
Epoch    4/20 Batch 2/3 Cost: 5.115668
Epoch    4/20 Batch 3/3 Cost: 3.430770
Epoch    5/20 Batch 1/3 Cost: 5.220237
Epoch    5/20 Batch 2/3 Cost: 3.468729
Epoch    5/20 Batch 3/3 Cost: 1.697594
Epoch    6/20 Batch 1/3 Cost: 3.449394
Epoch    6/20 Batch 2/3 Cost: 1.917554
Epoch    6/20 Batch 3/3 Cost: 10.997640
Epoch    7/20 Batch 1/3 Cost: 2.558268
Epoch    7/20 Batch 2/3 Cost: 5.090144
Epoch    7/20 Batch 3/3 Cost: 3.333734
Epoch    8/20 Batch 1/3 Cost: 7.939080
Epo